## Livrable 1 : Données
Bonjour et bienvenue dans le test de Notebook

Test d'intégration du code python dans un noteBook


In [1]:
print('Hello world !')

Hello world !


5 - Accès au données depuis Python


1. Afficher le nombre de documents de la collection

In [2]:
from pymongo import MongoClient

client = MongoClient('localhost', 27017)
db = client['DataProject']
collection_traffic = db['vehicules']

print(collection_traffic.find().count())

0


C:\Users\benja\anaconda3\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  import sys


2. Afficher les 5 premiers passages de véhicules de la collection.



In [3]:
from pymongo import MongoClient
import pprint

client = MongoClient('localhost', 27017)
db = client['DataProject']
collection_traffic = db['vehicules']

pprint.pprint(list(collection_traffic.find()[0:5]))

[]


3. Insérer un document avec un numéro d’arête de 501, puis le supprimer

In [4]:
from pymongo import MongoClient

client = MongoClient('localhost', 27017)
db = client['DataProject']
collection_traffic = db['vehicules']

collection_traffic.insert_one({'num_arete': 501})
print(collection_traffic.find_one({'num_arete': 501}))
collection_traffic.delete_one({'num_arete': 501})
print(collection_traffic.find_one({"num-arete": 501}))

{'_id': ObjectId('5ed65275f924a2599d801bd8'), 'num_arete': 501}
None


4. Afficher le nombre total de passages de véhicules par plage horaire :
    - En utilisant une boucle for.
    - En utilisant la fonction aggregate.

En utilisant une boucle `for`

In [5]:
from pymongo import MongoClient

client = MongoClient('localhost', 27017)
db = client['DataProject']
collection_traffic = db['vehicules']

vehicule_par_plage = {'m': 0, 's': 0}
for trafic in collection_traffic.find({'$or':[{'plage_horaire':'m'},  {'plage_horaire':'s'}]}):
    plage = trafic['plage_horaire']
    vehicule_par_plage[plage] += trafic['nb_vehicules']

print(vehicule_par_plage)

{'m': 0, 's': 0}


En utilisant la fonction `AGGREGATE`

In [6]:
from pymongo import MongoClient
import pprint

 #Connection a la base MongoDB
client = MongoClient('localhost', 27017)
db = client['DataProject']
collection_traffic = db['vehicules']

#Arguments de l'aggregate
arg = [{'$group': {'_id': '$plage_horaire', 'vehicule_par_plage': {'$sum': '$nb_vehicules'}}}]

#Affichage du resultat
pprint.pprint(list(collection_traffic.aggregate(arg)))

[]


6 - Convertir les données

In [7]:
from pymongo import MongoClient
import datetime

client = MongoClient('localhost', 27017)
db = client['DataProject']
collection_traffic = db['vehicules']

if db['vehicules_stamped'].count() <= 0:
    for trafic in collection_traffic.find():
        h = round(trafic['num_periode'] / 60)
        m = trafic['num_periode'] % 60
        j = trafic['num_jour'] + 1

        db['vehicules_stamped'].insert_one({'num_arete': trafic['num_arete'], 'date': datetime.datetime(2020, 1, j, h, m).strftime("%d/%m/%Y %Hh%Mm"),
                                                        'nb_vehicules': trafic['nb_vehicules']})
else:
    print('La base est déjà remplie')

C:\Users\benja\anaconda3\lib\site-packages\ipykernel_launcher.py:8: DeprecationWarning: count is deprecated. Use estimated_document_count or count_documents instead. Please note that $where must be replaced by $expr, $near must be replaced by $geoWithin with $center, and $nearSphere must be replaced by $geoWithin with $centerSphere
  


6.2 Validation des résultats

1. Compter le nombre de resultats

In [8]:
from pymongo import MongoClient

client = MongoClient('localhost', 27017)
db = client['DataProject']
collection_stamped = db['vehicules_stamped']

print(collection_stamped.count_documents({}))

0


2. Afficher les dates et les heures présentes dans la nouvelle collection.


In [9]:
from pymongo import MongoClient
import pprint

client = MongoClient('localhost', 27017)
db = client['DataProject']
collection_stamped = db['vehicules_stamped']

pprint.pprint(list(collection_stamped.find({}, {'date': 1})[0:5]))

[]


3. Afficher le nombre de passages de véhicules par tranche horaire.

In [10]:
from pymongo import MongoClient
import datetime

client = MongoClient('localhost', 27017)
db = client['DataProject']
collection_traffic = db['vehicules_stamped']

vehicule_par_plage = {'m': 0, 's': 0}
for trafic in collection_traffic.find():
    date = datetime.datetime.strptime(trafic['date'], "%d/%m/%Y %Hh%Mm")
    if datetime.time(7, 0, 0, 0) <= date.time() < datetime.time(10, 0, 0, 0):
        vehicule_par_plage['m'] += trafic['nb_vehicules']
    else:
        vehicule_par_plage['s'] += trafic['nb_vehicules']

print(vehicule_par_plage)


{'m': 0, 's': 0}


In [ ]:
from pymongo import MongoClient
import datetime

client = MongoClient('localhost', 27017)
db = client['DataProject']
collection_traffic = db['vehicules_stamped']

vehicule_par_plage = {'m': 0, 's': 0}
for trafic in collection_traffic.find():
    date = datetime.datetime.strptime(trafic['date'], "%d/%m/%Y %Hh%Mm")
    if datetime.time(7, 0, 0, 0) <= date.time() < datetime.time(10, 0, 0, 0):
        vehicule_par_plage['m'] += trafic['nb_vehicules']
    else:
        vehicule_par_plage['s'] += trafic['nb_vehicules']

print(vehicule_par_plage)


2. Afficher les dates et les heures présentes dans la nouvelle collection.


In [ ]:
from pymongo import MongoClient
import pprint

client = MongoClient('localhost', 27017)
db = client['DataProject']
collection_stamped = db['vehicules_stamped']

pprint.pprint(list(collection_stamped.find({}, {'date': 1})[0:5]))

3. Afficher le nombre de passages de véhicules par tranche horaire.

In [ ]:
from pymongo import MongoClient
import datetime

client = MongoClient('localhost', 27017)
db = client['DataProject']
collection_traffic = db['vehicules_stamped']

vehicule_par_plage = {'m': 0, 's': 0}
for trafic in collection_traffic.find():
    date = datetime.datetime.strptime(trafic['date'], "%d/%m/%Y %Hh%Mm")
    if datetime.time(7, 0, 0, 0) <= date.time() < datetime.time(10, 0, 0, 0):
        vehicule_par_plage['m'] += trafic['nb_vehicules']
    else:
        vehicule_par_plage['s'] += trafic['nb_vehicules']

print(vehicule_par_plage)
